In [28]:
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output, Image, display, HTML
import msgpack as mp
import msgpack_numpy as mpn
import base64
from datetime import datetime
import os
from tqdm import tqdm
from datetime import timedelta
import keyboard

In [29]:
base_pth = "C:\mira\splitVideos\SAMXXXXXXXXU010120000000XXXXXXXXX\Session 04-02-22_14-03-00_359"
_prm = os.listdir(r"C:\mira\splitVideos\SAMXXXXXXXXU010120000000XXXXXXXXX\Session 04-02-22_14-03-00_359")

for i in _prm:
    if i.startswith("PARAMS"):
        _p_fname = i
prm_file = base_pth + "\\" + _p_fname
prm_file


p = open(prm_file, "rb")
unpacker = mp.Unpacker(p, object_hook=mpn.decode)
prm = []
for unpacked in unpacker:
    prm.append(unpacked)
prm.pop(0)
prm.pop(0)


video_time = []
for i in prm:
    video_time.append(datetime.strptime(i[0], '%Y-%m-%d %H:%M:%S.%f'))
v_start = video_time[0]
v_start


"""for df"""
video_time_df = []
for i in prm:
    video_time_df.append(i[0])



In [30]:
video_data = cv2.VideoCapture(base_pth+"//Video.avi")

_i = 0
frames = []
while video_data.isOpened:
    
    ret, frame = video_data.read()
    if not ret:
        break
    frames.append(frame)
    frame = cv2.flip(frame, 1)
    vid_diff = video_time[_i] - video_time[0] 
    _vid_diff = vid_diff.total_seconds()
    _i += 1
    # font
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    # org
    org = (50, 50)
    
    # fontScale
    fontScale = 1
    
    # Blue color in BGR
    color = (255, 0, 0)
    
    # Line thickness of 2 px
    thickness = 2
    
    # Using cv2.putText() method
    image = cv2.putText(frame, str(_vid_diff), org, font, 
                    fontScale, color, thickness, cv2.LINE_AA)
    if ret:
        cv2.imshow("",frame)
        # plt.show()
        
    if cv2.waitKey(25) & 0xFF == ord('q'):
          break
    
    
    
    
cv2.destroyAllWindows()
        

In [31]:
imu_1 = pd.read_csv(base_pth+"//1_imu.csv")
imu_2 = pd.read_csv(base_pth+"//2_imu.csv")

In [32]:
def datetime_to_s(df, c_name, sys_t=None, offset=0):
    """this function changes datetime string to seconds"""
    inf = datetime.strptime(df[c_name][0], '%Y-%m-%d %H:%M:%S.%f')
    dif_list = []
    for idx, val in enumerate(df[c_name]):
        _val = datetime.strptime(val, '%Y-%m-%d %H:%M:%S.%f')
        if sys_t:
            # diff = (_val - inf) - sys_t
            diff = _val - sys_t

        else:
            diff = _val - inf
            
        _diff = diff.total_seconds()
        # df["seconds"].loc[idx] = _diff
        dif_list.append(_diff)
    df["sys_sec"] = dif_list
    return df

In [35]:
# plt.plot(imu_1["imu_time"], imu_1["ax"])
imu_1 = datetime_to_s(imu_1, "sys_time", v_start)
imu_2 = datetime_to_s(imu_2, "sys_time", v_start)

"""for df"""

imu_1df = pd.read_csv(base_pth+"//1_imu.csv", parse_dates=['sys_time'], index_col='sys_time')
imu_1df.index.name = "time"
imu_2df = pd.read_csv(base_pth+"//2_imu.csv", parse_dates=['sys_time'], index_col='sys_time')
imu_2df.index.name = "time"
vid_df = pd.DataFrame(video_time_df, columns=["time"])
vid_df = pd.to_datetime(vid_df["time"])
# imu_1df

In [40]:
# plt.plot(imu_1["sys_sec"], imu_1["az"])
# imu_1df

In [38]:
imutime = [] 
for t in tqdm(vid_df[:len(frames)-1]):
    print(t)
    imutime.append(imu_1df.index[imu_1df.index.get_loc(t, method='nearest')])
grps = []
for t in tqdm(imutime):
    grps.append(imu_1df.between_time((t - timedelta(seconds=2)).time(), (t + timedelta(seconds=1)).time()))

  0%|          | 0/745 [00:00<?, ?it/s]

2022-02-04 14:04:22.302062


InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [ ]:
start_at = 3500
i=0

for frame, t, dat in zip(frames[start_at+1:], imutime[start_at:], grps[start_at:]):
    if keyboard.is_pressed('q'):
        plt.close()
        break
    else:
        plt.clf()
        plt.subplot(121)
        plt.title(i)
        plt.imshow(frame)
        plt.subplot(322)
        plt.title(t)
        plt.plot(dat.ax)
        plt.axvline(t, color='r')
        plt.ylim(imu_1df.ax.min(), imu_1df.ay.max())
        plt.subplot(324)
        plt.plot(dat.ay)
        plt.axvline(t, color='r')
        plt.ylim(imu_1df.ay.min(), imu_1df.ay.max())
        plt.subplot(326)
        plt.plot(dat.az)
        plt.axvline(t, color='r')
        plt.ylim(imu_1df.az.min(), imu_1df.az.max())
        i+=1
        plt.pause(0.0001)